## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [1]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [2]:
# Always remember to do this!
load_dotenv(override=True)

True

In [3]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

OpenAI API Key not set
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_
OpenRouter API Key exists and begins sk-


In [4]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [5]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [17]:
optimized_messages = messages = [{"role": "user", "content": "Give me one deeply challenging question to test LLM intelligence. Only output the question."}]

openai = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_api_key,
)
response = openai.chat.completions.create(
    # model="gpt-5-mini",
    model="qwen/qwen-2.5-72b-instruct",
    messages=messages,
    # messages=[{"role": "user", "content": "1+1"}],
    max_tokens=100
)
question = response.choices[0].message.content
print(question)


Can you design an algorithm that not only solves any given Rubik's Cube in the minimum number of moves but also explains the underlying mathematical principles and group theory concepts that make the solution optimal?


In [18]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

## Note - update since the videos

I've updated the model names to use the latest models below, like GPT 5 and Claude Sonnet 4.5. It's worth noting that these models can be quite slow - like 1-2 minutes - but they do a great job! Feel free to switch them for faster models if you'd prefer, like the ones I use in the video.

In [19]:
# The API we know well
# I've updated this with the latest model, but it can take some time because it likes to think!
# Replace the model with gpt-4.1-mini if you'd prefer not to wait 1-2 mins

model_name = "gpt-5-nano"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Short answer: Yes. You can build an optimal Rubik’s Cube solver that guarantees a minimum-move solution for any scramble, and you can accompany it with a solid explanation of the group-theory ideas that make optimality possible. In practice this is done with a combination of pattern databases (PDBs) and a search procedure that is guaranteed not to overestimate the remaining distance (e.g., IDA* with an admissible heuristic). Below is a concrete high-level design you can implement or study, plus the mathematical background that makes it work.

1) What “minimum-move solver” means in Rubik’s Cube terms
- The cube group G is generated by the six face turns (U, D, L, R, F, B). A solution is a shortest sequence of these moves that transforms a given scramble to the solved state.
- The “minimum number of moves” depends on the move metric you use. The common one is the quarter-turn metric (QTM): a 90° turn counts as 1, a 180° turn counts as 2. In that metric, the maximal distance (God’s number) is known to be 20 for a 3×3×3 cube. (If you use other metrics, the numbers differ.)
- An optimal solver guarantees that the found sequence length equals the true shortest distance from the start state to the solved state.

2) Core ideas you’ll need to guarantee optimality
- Admissible heuristic: A lower-bound estimate h(s) of the remaining moves from a state s to solved. It never overestimates the true distance. If you use a heuristic that is always ≤ the true distance, an A*/IDA* search will never discard an optimal path.
- Pattern databases (PDBs): Precomputed exact distances for sub-patterns of the cube. Each PDB stores the minimal moves needed to solve a particular subset of cubies (e.g., a subset of edges, a subset of corners, or their orientations). The collection of these distances, combined in an admissible way, gives a very strong lower bound.
- Additive heuristics: When you partition the cube into disjoint sub-patterns, you can often add their distances to get a tighter lower bound (as long as the sub-patterns are chosen so that their solved states are compatible with the whole-cube goal and moves don’t create over-counting). This is the standard way PDBs are used for optimal solvers.
- State encoding and pruning: Efficient representation of cube states, move tables for fast transition on a computer, and pruning rules (e.g., don’t immediately undo the previous move).

3) The algorithm at a high level
- State representation
  - Represent the cube by the permutation/orientation of 8 corners and 12 edges. A compact encoding is common (e.g., indices for corner permutation + orientation and edge permutation + orientation). There are parity/orientation constraints you can enforce to keep the state space valid.
- Precomputation (offline)
  - Build move tables for the six basic moves (and possibly their 180° versions). This speeds up applying moves to states.
  - Build one or more pattern databases (PDBs) for disjoint sub-patterns (e.g.,:
    - Corner orientation (how far corners are from oriented),
    - Corner permutation (how far corners are from the solved permutation),
    - Edge orientation,
    - Edge permutation,
    - Or more elaborate combined patterns).
  - Ensure the PDBs used give an admissible bound when combined (e.g., take the maximum or a sum of distances depending on the disjointness of patterns).
- Search (online)
  - Use IDA* (iterative deepening A*) with the PDB-based heuristic h(s).
  - Bound-setting loop:
    - bound = h(start)
    - while true:
      - result = depth-first search(state=start, g=0, bound, last_move=None)
      - if result found a solution path, return it
      - bound = next_bound_from_dfs(results)  // the smallest f = g + h that exceeded the previous bound
  - DFS routine:
    - If g + h(state) > bound, prune and report the threshold
    - If state is solved, return success
    - Generate moves in some order (often using a small set of generators; skip moves that are immediately inverse of the previous move to prune)
    - For each move, update state via the move, recurse with g+1, and record the path if a solution is found
  - Optional speedups:
    - Transposition table (hash table) to avoid re-expanding the same state at the same or higher g
    - Symmetry pruning (exploit cube symmetries to reduce duplicates)
    - Two-phase scheme (see below) if you want a practical fast solver that is easy to implement, but note it is typically not guaranteed to be optimal unless you adapt it to an optimal, PDB-based form.
- Optional: two-phase structure for practicality
  - Phase 1 moves the state into a subgroup H of the cube group (e.g., a set of states with certain invariants like edge orientation and corner orientation fixed in some sense).
  - Phase 2 solves within H using a restricted move set.
  - A strictly optimal version of the two-phase approach uses phase-1 and phase-2 distances from PDBs and a search that guarantees you reach the global optimum; this is essentially an optimal variant of the classic Thistlethwaite-style approach with added pattern databases.

4) The math behind the design (group-theory essentials)
- The Rubik’s Cube group G
  - G is the group generated by the six face-turns U, D, L, R, F, B. Each element is a sequence of moves, reduced by the relations (e.g., U^4 = I, etc.).
  - The order of G is enormous (on the order of 4.3×10^19), but for algorithm design you don’t need to enumerate all of it—only explore a small, relevant portion from a given scramble.
- Piece structure and invariants
  - Corner pieces: 8 pieces with 3 possible orientations each (except the orientation constraint). This gives 8! permutations × 3^7 usable orientations (the last corner’s orientation is determined by the others).
  - Edge pieces: 12 pieces with 2 possible orientations each (except the orientation constraint). This gives 12! × 2^11 usable states.
  - Parity constraints: The parity (even/odd) of the corner permutation matches the parity of the edge permutation. The sum of corner orientations mod 3 is 0, and the sum of edge orientations mod 2 is 0.
  - These constraints define the reachable portion of the full 8!·3^8 × 12!·2^12 state space.
- Generators, relations, and structure
  - The six generators (U, D, L, R, F, B) and their relations create a non-abelian group with many hidden symmetries. Understanding that the same move can be represented by many different sequences leads to the importance of reduction and pruning.
- Subgroups, stabilizers, and phase ideas
  - A common optimal-solver strategy uses a stabilizer-like approach: first move the cube into a subgroup where a large portion of the cube is already “in place” or oriented in a constrained way, then finish with a smaller, more manageable search.
  - The two-phase idea (Thistlethwaite/Kociemba lineage) follows a similar idea: reduce to a subgroup with the remaining moves, then complete with a restricted move set. For guaranteed optimality, you combine this with exact distance data from PDBs in both phases.
- Pattern databases as exact subproblems
  - A PDB stores, for a subset of cubies, the exact distance from each possible projection to the solved projection.
  - If the subsets are chosen carefully (often disjoint in a way compatible with the move set), the distances can be added (or otherwise combined) to yield a valid lower bound for the full cube. This lower bound is admissible because any complete solution must also solve those sub-patterns.
  - The sum (or max, depending on the partitioning) of these sub-pattern distances never overestimates the true distance, which guarantees optimality when used in a complete search like IDA*.

5) A practical blueprint you can implement or study
- Data structures
  - A compact state encoding: a small struct with corner permutation (8 elements), corner orientation (3^7 states), edge permutation (12 elements), edge orientation (2^11 states).
  - Move tables: precompute the effect of U, D, L, R, F, B on the corner/edge permutations and orientations to speed up transitions.
  - PDBs: offline precompute several pattern databases for disjoint sub-patterns (e.g., corners only, edges only, combined but disjoint groups). Store them as hash tables or compact arrays keyed by a pattern index.
  - Transposition table: a hash table mapping state-identity to the best g seen so far (to prune repeats within a search).
- Online search procedure
  - Heuristic h(s): compute h from PDBs. A common approach is to take h = max(h_corner(s), h_edge(s), h_corner_perm(s), h_edge_perm(s)) or a sum of several compatible sub-pattern distances if your partitioning supports additivity.
  - IDA* loop:
    - bound = h(start)
    - while true:
      - result = dfs(start, g=0, bound, last_move=None)
      - if result found solution path: return it
      - bound = next_bound encountered during dfs (the smallest f = g + h that exceeded the previous bound)
  - DFS details:
    - If g + h(state) > bound, return the value for updating bound.
    - If state is solved, return success.
    - Generate moves in a sensible order (e.g., avoid consecutive inverses, prefer moves that improve the heuristic).
    - Use transposition table to prune repeats and avoid cycles.
  - Optional improvements:
    - Bidirectional search in a bound-aware way (more complex but can dramatically reduce search effort).
    - Move-ordering heuristics that prioritize moves likely to reduce the PDB distances.
- Practical considerations
  - Building and storing PDBs is the heavyweight part. They may require gigabytes of memory and some offline computation, but they enable strong, admissible heuristics.
  - An off-the-shelf optimal solver would typically use a multi-PDB setup with IDA* (or an A*-like approach) to guarantee optimality. There are public solvers and libraries you can study to see concrete implementations (e.g., academic papers and open-source projects on optimal Rubik’s Cube solvers).

6) A little math-oriented intuition you can share
- You can illustrate optimality with a small example without full-blown computation:
  - Use a commutator to move a single piece while leaving others largely undisturbed: [R, U] = R U R^-1 U^-1 yields a cube move that affects a small subset of cubies in a structured way. Such commutator sequences are the building blocks of many optimal sub-solutions.
  - The pattern databases encode exact distances for subpatterns; any full solution must, when projected to each subpattern, be at least as long as the subpattern’s distance. Hence the maximum (or sum) of subpattern distances is a valid lower bound, guaranteeing that the search never discards a truly optimal path.
- The parity/orientation constraints you enforce during state generation are direct consequences of the group structure of the cube and the generator set. Any consistent solver must respect those invariants.

7) What this looks like in practice
- Bottom line: an optimal solver is feasible and well-studied. The standard, well-documented methods combine:
  - A formal state-space model of the cube (corners/edges with permutation and orientation),
  - Offline pattern databases for strong, admissible heuristics,
  - An optimal search (IDA* or A*) with careful pruning and transposition-table support.
- If you want to start quickly, consider:
  - Studying Korf’s work on optimal Rubik’s Cube solving with pattern databases (pioneering in this area),
  - Looking at existing open-source Rubik’s Cube solvers that implement optimal or near-optimal strategies with PDBs (to see practical data structures and code layout),
  - Building a simplified version first (e.g., only a subset of cubies or a smaller puzzle) to understand the interaction between group theory and search heuristics.

8) If you want, I can provide
- A concrete pseudo-code outline for an IDA*-based optimal solver with a multi-PDB heuristic,
- A concrete, compact state representation and move-application routine,
- Example discussions of a couple of commutator sequences to illustrate how you can move individual pieces with controlled side effects,
- Or pointers to literature and open-source projects you can study to see full implementations.

Would you like me to draft a concrete pseudo-code skeleton (state encoding, move tables, PDB interface, and the IDA* search loop) you can implement in a language of your choice? Or would you prefer a deeper, more theory-first write-up with worked examples of the group-theory components and simple commutator sequences?

In [ ]:
# ===== ORIGINAL CODE USING ANTHROPIC/CLAUDE API =====
# Anthropic has a slightly different API, and Max Tokens is required
# model_name = "claude-sonnet-4-5"
# claude = Anthropic()
# response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
# answer = response.content[0].text
# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)

# ===== USING FREE LLM API INSTEAD =====
import sys
sys.path.append('..')
from freellm_integration import FreeLLMClient

model_name = "FreeLLM API (apifreellm.com)"

# FreeLLM API expects a simple message string, not the messages array format
user_message = messages[0]["content"]

# Use the FreeLLM client
async with FreeLLMClient() as client:
    response = await client.chat(user_message)
    answer = response.response

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Below is an outline of a “minimum-move” Rubik’s Cube solver based on group-theoretic ideas and heuristic search (the so-called Kociemba two-phase algorithm).  We break the discussion into

 1. Definitions and group-theoretic setup  
 2. State‐space representation  
 3. The two‐phase algorithm  
 4. Heuristic (pattern‐database) pruning  
 5. Why this finds (near) minimum‐length solutions  

—everything with KaTeX for clarity.  

1.  Definitions and group-theoretic setup  
——————————————————  
•  Let \(G\) be the Rubik’s Cube group: the set of all reachable cube states under face turns, with group operation “do one state then the next.”  
•  We fix a generating set \(S=\{U,D,L,R,F,B\}\), each letter a 90° clockwise face turn (we also allow their inverses).  
•  The solved cube is the identity element \(e\in G\).  
•  A subgroup \(H\le G\) is chosen so that cosets \(G/H\) capture certain “reduced” states that are easier to solve.  

Group-theoretic principle: we write each full solution as  
\[
   g = p\cdot q,\quad p\in G,\;q\in H,
\]
where phase-1 brings the cube into \(H\), and phase-2 solves it from there.  

2.  State-space representation  
——————————————————  
We represent a cube state by two arrays:  
 (a) corner permutation \(\pi_c\) of length 8 and corner orientation \(\kappa_c\in\{0,1,2\}^8\).  
 (b) edge   permutation \(\pi_e\) of length 12 and   edge   orientation \(\kappa_e\in\{0,1\}^{12}\).  

Together this is a 4-tuple  
\[
   \bigl(\pi_c,\kappa_c,\pi_e,\kappa_e\bigr)
   \;=\; \text{state in }G.
\]

3.  The two-phase algorithm  
——————————————————  
Phase 1:  
– Goal: find a sequence \(p\in G\) so that \(p\) brings the cube into a subgroup \(H\) in which all edge-flip and corner-twist conditions are solved and the middle‐layer edges lie in the middle layer.  
– We search in the coset graph of \(G/H\).  

Phase 2:  
– Starting from \(p(\text{scrambled})\), we search inside \(H\) to reach the identity \(e\).  

Outline in pseudocode:  
\[
\begin{aligned}
   &\text{INPUT: scrambled state }s_0. \\
   &\text{FOR depth }d_1 = 0,1,2,\dots,D_1: \\
   &\quad\text{search Phase-1 depth-first up to }d_1\text{ to find }p\text{ such that }p(s_0)\in H.\\
   &\quad\text{IF success at }d_1\text{, let }s_1 = p(s_0).\\
   &\quad\text{FOR depth }d_2 = 0,1,2,\dots,D_2: \\
   &\qquad\text{search Phase-2 depth-first up to }d_2\text{ to solve }s_1\text{ in }H.\\
   &\qquad\text{IF success, return }p\cdot q\text{ (full solution).}
\end{aligned}
\]

4.  Heuristic (pattern-database) pruning  
——————————————————  
To make the depth-first search practical, we precompute “pattern databases” that give admissible lower bounds on the number of moves to reach the phase-1 or phase-2 goal.  For example:  
•  A corner database storing  
  \(\;h_c(\pi_c,\kappa_c)=\) min moves to solve corners.  
•  An edge database storing  
  \(\;h_e(\pi_e,\kappa_e)=\) min moves to solve edges.  

Then at any state \(x\) during search,  
\[
   h(x)=\max\bigl(h_c(\text{corners of }x),\,h_e(\text{edges of }x)\bigr)
\]
is an admissible heuristic – it never overestimates the true distance.  We prune any branch whose current depth \(+\;h(x)\) exceeds our depth limit.  

5.  Why this finds (near) minimum‐length solutions  
————————————————————————————————  
•  The two‐phase decomposition using subgroup \(H\) reduces the effective branching factor.  
•  IDA* (iterative-deepening A*) with the admissible heuristic guarantees minimality *within each phase* under our move metric.  
•  Combined, the found sequence \(p\cdot q\) is usually very close to the true “God’s‐number” minimum (which is ≤ 20 moves in the quarter‐turn metric).  

Mathematical principle: we exploit  
– the group structure (\(G,H\), cosets) to split the huge search into manageable sub-searches,  
– admissible heuristics from pattern databases to prune sharply,  
– IDA* to guarantee optimality for the given depth limit.  

References for full implementation details:  
• Herbert Kociemba, “A Two‐Phase Algorithm for the Rubik’s Cube,” Cubism Ltd., 1992.  
• Morwen Thistlethwaite, “Cube‐Solving,” 1981 (subgroup decomposition).  

This combination of group-theoretic decomposition and heuristic search yields extremely short solutions (often optimal or near‐optimal) in milliseconds on modern hardware.

In [21]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing an algorithm that not only solves a Rubik's Cube in the minimum number of moves but also explains the underlying mathematical principles and group theory concepts is a fascinating and complex task. The "minimum number of moves" aspect points directly to algorithms based on God's Number, primarily **Kociemba's Algorithm** or variations, which are the most practical approaches to finding near-optimal or optimal solutions.

Let's break this down into the algorithm design and the mathematical explanation.

---

## Algorithm Design: Optimal Rubik's Cube Solver (Kociemba's Algorithm)

The most widely used algorithm for finding near-optimal solutions (often matching God's Number, which is 20 half-turn moves for any state) is a two-phase algorithm developed by Herbert Kociemba, based on ideas from Thistlethwaite's algorithm. It leverages group theory by breaking the problem into a search through subgroups.

### 1. Cube Representation

First, we need a way to represent the cube's state computationally.
A standard approach is to represent the 8 corners and 12 edges independently.

*   **Corners:** Each corner piece has a specific position (index 0-7) and an orientation (0, 1, or 2, indicating how it's twisted relative to its solved state).
    *   `cp[8]`: permutation of corner positions (0-7)
    *   `co[8]`: orientation of corners (0, 1, 2)
*   **Edges:** Each edge piece has a specific position (index 0-11) and an orientation (0 or 1, indicating if it's flipped).
    *   `ep[12]`: permutation of edge positions (0-11)
    *   `eo[12]`: orientation of edges (0, 1)

These representations are subject to certain group theory constraints (e.g., sum of `co` mod 3 is 0, sum of `eo` mod 2 is 0, parity of `cp` equals parity of `ep`).

### 2. Precomputation (Crucial for Speed and Optimality)

Kociemba's algorithm relies heavily on precomputed lookup tables, often called **pruning tables** or **distance tables**. These tables store the minimum number of moves required to reach a specific "sub-goal state" from any given state within a reduced problem space. They act as heuristics for the search algorithm.

The state space is too large for a single table, so it's broken down:

*   **Phase 1 Pruning Tables:**
    *   `P1_table_co`: `corner_orientation` -> min_moves (3^7 states)
    *   `P1_table_eo`: `edge_orientation` -> min_moves (2^11 states)
    *   `P1_table_slice`: `middle_slice_edge_positions` -> min_moves (12 choose 4 states)
    *   These three can be combined into one larger table, e.g., `(co_idx, eo_idx, slice_idx) -> min_moves`.
*   **Phase 2 Pruning Tables:**
    *   `P2_table_cp_ep`: `(corner_permutation, edge_permutation_in_slice)` -> min_moves (8! * 4! states)
    *   `P2_table_ep_ep`: `(edge_permutation_of_remaining, edge_permutation_of_middle_slice)` -> min_moves (8! * 4! states)
    *   These are for specific permutations *within* the G1 subgroup.

These tables are generated using a Breadth-First Search (BFS) backwards from the solved state (or a state in the target subgroup).

### 3. The Two-Phase Algorithm

Kociemba's algorithm divides the solution into two phases, moving from the full Rubik's Cube Group (G0) to a specific subgroup (G1), and then from G1 to the solved state.

**Phase 1: Orienting Edges/Corners & Placing M-Slice Edges (Goal: Reach Subgroup G1)**

*   **Goal State:** All corners are oriented correctly (co=0), all edges are oriented correctly (eo=0), and the 4 middle-slice edges (E_M) are in the middle slice (i.e., their positions are among a specific 4 slots). The actual permutation of pieces doesn't matter yet.
*   **Allowed Moves:** Only quarter turns on the R, L, F, B faces (R, R', R2, L, L', L2, etc.) and half turns on the U, D faces (U2, D2). This restricted move set is key to making G1 a subgroup and simplifying the search space.
*   **Search Algorithm:** Iterative Deepening A* (IDA*) search.
    *   It starts with a depth limit `d=0` and incrementally increases `d`.
    *   For each state `s`, it calculates a heuristic `h(s)` using the precomputed Phase 1 pruning tables (e.g., `h(s) = max(P1_table_co[co_idx], P1_table_eo[eo_idx], P1_table_slice[slice_idx])`).
    *   It explores states as long as `current_depth + h(s) <= d`.
    *   When a state in G1 is reached (i.e., `co=0`, `eo=0`, `middle_slice_edges_in_place`), Phase 1 is complete.

**Phase 2: Permuting Pieces within G1 (Goal: Reach Solved State)**

*   **Goal State:** The completely solved cube.
*   **Allowed Moves:** All 18 standard moves (R, R', R2, L, L', L2, F, F', F2, B, B', B2, U, U', U2, D, D', D2). However, applying these moves *from a state in G1* must *maintain the G1 property*. This means the orientation and slice properties are preserved by these moves.
*   **Search Algorithm:** Another IDA* search.
    *   The initial state for Phase 2 is the G1 state found at the end of Phase 1.
    *   The heuristic `h(s)` for Phase 2 uses the precomputed Phase 2 pruning tables (e.g., `h(s) = max(P2_table_cp_ep[cp_idx][ep_idx], ...) `).
    *   The search explores paths until the fully solved state is reached.

### 4. Overall Algorithm Steps

1.  **Initialize:** Define the cube's initial scrambled state (`initial_state`).
2.  **Precompute:** Generate all necessary pruning tables (this is a one-time, significant computation).
3.  **Iterate Phase 1 Length:** For `len1` from `0` to `max_phase1_len` (e.g., 12-14 moves):
    *   **IDA* Search (Phase 1):** Find all move sequences of length `len1` that transform `initial_state` into a state belonging to the G1 subgroup.
        *   Use the Phase 1 pruning tables to guide the search.
        *   For each `g1_state` found:
            *   **Iterate Phase 2 Length:** For `len2` from `0` to `max_phase2_len` (e.g., 12-14 moves):
                *   **IDA* Search (Phase 2):** Find the shortest move sequence of length `len2` that transforms `g1_state` into the `solved_state`, using the allowed moves for Phase 2.
                    *   Use the Phase 2 pruning tables to guide the search.
                    *   If a solution is found (a sequence `S2`), then `S1 + S2` is a potential solution.
                *   If `len1 + len2` is the shortest found so far, store it.
                *   Optimization: If a solution of length `L` is found, you can stop searching `len1+len2` if it exceeds `L`.
    *   The algorithm stops once the first solution is found, as IDA* guarantees finding the shortest path for the combined length.

### Example Solution Structure

A solution might look like:
`R U' F2 B' L2 D2 R2 U2 F2 L2` (Phase 1: brings to G1)
`R' U F D' R L U' F' B D R'` (Phase 2: solves from G1)
Total: 20 moves.

---

## Explaining the Underlying Mathematical Principles and Group Theory Concepts

The Rubik's Cube is a quintessential example of group theory in action.

### 1. The Rubik's Cube Group (G)

*   **Definition of a Group:** A set `S` with a binary operation `*` satisfying four axioms:
    1.  **Closure:** For all `a, b` in `S`, `a * b` is in `S`.
    2.  **Associativity:** For all `a, b, c` in `S`, `(a * b) * c = a * (b * c)`.
    3.  **Identity Element:** There exists an `e` in `S` such that for all `a` in `S`, `a * e = e * a = a`.
    4.  **Inverse Element:** For all `a` in `S`, there exists an `a⁻¹` in `S` such that `a * a⁻¹ = a⁻¹ * a = e`.
*   **Rubik's Cube as a Group:**
    *   **Elements:** Each possible scrambled state of the Rubik's Cube is an element. The "solved" state is the identity element.
    *   **Operation:** Applying a sequence of moves (e.g., `R U F'`) is the group operation.
    *   **Closure:** Applying one sequence of moves followed by another results in a new valid cube state.
    *   **Associativity:** The order of grouping move sequences doesn't matter (e.g., `(R U) F = R (U F)`).
    *   **Identity:** Doing nothing (or applying an empty sequence of moves) leaves the cube in the same state.
    *   **Inverse:** Every move (e.g., `R`) has an inverse (`R'`) that undoes it. Every sequence of moves also has an inverse that restores the cube to its previous state.
*   **Order of the Group:** The total number of possible distinct configurations of the Rubik's Cube. This is approximately `4.3 * 10^19` (43 quintillion), which is why brute-force search is impossible.

### 2. Permutations and Orientations

The pieces of the Rubik's Cube undergo transformations.

*   **Permutations:**
    *   When you turn a face, the corner pieces (cubies) are permuted among the 8 corner positions, and the edge pieces are permuted among the 12 edge positions.
    *   This is modeled by the **symmetric group** `S_n`, where `n` is the number of items being permuted. For corners, `S_8`; for edges, `S_12`.
    *   **Parity:** A permutation is either even or odd. For the Rubik's Cube, the parity of the corner permutation `P(cp)` must always be equal to the parity of the edge permutation `P(ep)`. This is a crucial invariant: you can't have an odd corner permutation and an even edge permutation.

*   **Orientations:**
    *   Corner pieces can be twisted in place (3 possible orientations relative to their "correct" state: 0, 1, 2).
    *   Edge pieces can be flipped (2 possible orientations: 0, 1).
    *   These are related to **modular arithmetic**:
        *   The sum of all 8 corner orientations (modulo 3) must always be 0. You can't twist just one corner.
        *   The sum of all 12 edge orientations (modulo 2) must always be 0. You can't flip just one edge.

These constraints (`P(cp) == P(ep)`, `sum(co)%3 == 0`, `sum(eo)%2 == 0`) mean that not all `8! * 12! * 3^8 * 2^12` theoretical states are reachable. The actual reachable state space is much smaller, precisely `(8! * 12! * 3^8 * 2^12) / (3 * 2 * 2) = 43,252,003,274,489,856,000`.

### 3. Subgroups and Cosets (The Heart of Kociemba's Algorithm)

The key to Kociemba's algorithm is to break down the large group `G0` into smaller, more manageable **subgroups**.

*   **Subgroup:** A subset of a group that is itself a group under the same operation.
*   **Kociemba's Subgroups:**
    *   **G0:** The full Rubik's Cube Group. All `4.3 * 10^19` states.
    *   **G1:** The subgroup of states where:
        1.  All corner orientations are correct (`co` are all 0).
        2.  All edge orientations are correct (`eo` are all 0).
        3.  The 4 edges of the M-slice (middle layer) are within the M-slice positions (though their permutation might be scrambled).
        *   **Why is G1 a subgroup?** The restricted moves in Phase 1 (R, L, F, B quarter-turns and U, D half-turns) *preserve* these properties. If you start in G1 and apply these moves, you stay in G1.
        *   The size of G1 is significantly smaller than G0 (approx. `6.6 * 10^10` states).
    *   **G2:** The identity element (the solved cube).

*   **Cosets:** A coset is a "shifted" version of a subgroup. If `H` is a subgroup of `G`, and `g` is an element of `G`, then `gH = {gh | h ∈ H}` is a left coset of `H` in `G`.
    *   **Phase 1's Role:** Phase 1 of Kociemba's algorithm starts from an arbitrary scrambled state `s` (an element in G0). The goal of Phase 1 is to find a sequence of moves `S1` such that `s * S1` (the state after applying `S1` to `s`) is an element of the subgroup G1. In other words, Phase 1 finds an element that belongs to the specific coset `s⁻¹G1` (or `G1s` as a right coset) of G1 in G0.
    *   **Phase 2's Role:** Once an element `g1_state` in G1 is reached, Phase 2 then finds a sequence of moves `S2` (using all moves, but ensuring the intermediate states remain within G1) that brings `g1_state` to the solved state (identity element). This is essentially solving the cube *within* the G1 subgroup.

This two-phase approach dramatically reduces the search space compared to searching the entire G0 space directly for an optimal solution. Instead of one massive search, it's two smaller, interdependent searches.

### 4. Search Algorithms and Heuristics

*   **IDA* (Iterative Deepening A*)**: This is a graph search algorithm that is both space-efficient (like Iterative Deepening Depth-First Search) and time-efficient (like A* search, using heuristics).
    *   **Heuristics:** A heuristic function `h(n)` estimates the cost from node `n` to the goal. For optimal solutions, the heuristic must be **admissible** (never overestimates the true cost) and preferably **consistent** (triangle inequality holds).
    *   **Pruning Tables as Heuristics:** The precomputed pruning tables provide the *exact* minimum number of moves to reach a specific sub-goal state (e.g., a specific edge orientation configuration) under certain move restrictions. These are perfect (admissible and consistent) heuristics for the sub-problems they represent. By combining the maximum of several such heuristics, IDA* efficiently prunes branches that cannot lead to an optimal solution within the current depth limit.

### 5. Optimality Guarantees

*   Kociemba's algorithm, when implemented correctly with exhaustive pruning tables and an IDA* search, is guaranteed to find *a* shortest path for the chosen `len1` and `len2` values.
*   By iterating `len1` and `len2` from 0 upwards, the *first* solution found will be the optimal solution in terms of `len1 + len2` (the total number of moves).
*   The "God's Number" (20 quarter-turns or 20 half-turns) represents the absolute minimum for any state. Kociemba's algorithm reliably finds solutions very close to this number, often achieving it. The computational cost of building the pruning tables and running the search is high but feasible on modern computers.

---

## Conclusion: The Elegance of Applied Math

The Rubik's Cube, initially appearing as a mere toy, hides a deep and beautiful mathematical structure. Kociemba's algorithm showcases how abstract group theory provides the fundamental framework for efficiently tackling problems with enormous state spaces. By understanding the symmetries, permutations, and subgroup relationships, we can decompose an intractable problem into manageable pieces, leading to elegant and optimal solutions, truly bridging the gap between abstract mathematics and practical problem-solving.

In [24]:
# ===== ORIGINAL CODE USING DEEPSEEK API =====
# deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
# model_name = "deepseek-chat"
# response = deepseek.chat.completions.create(model=model_name, messages=messages)
# answer = response.choices[0].message.content
# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)

# ===== USING FREE LLM API INSTEAD =====
import sys
sys.path.append('..')
from freellm_integration import FreeLLMClient

model_name = "FreeLLM API (apifreellm.com)"

# FreeLLM API expects a simple message string, not the messages array format
user_message = messages[0]["content"]

# Use the FreeLLM client
async with FreeLLMClient() as client:
    response = await client.chat(user_message)
    answer = response.response

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

### Algorithm Overview:
**Optimal Rubik's Cube Solver with Mathematical Explanation**

1. **Input**: A scrambled Rubik's Cube state.  
2. **Output**: A sequence of moves solving the cube optimally, along with explanations of group theory principles.

---

### Steps:

#### **1. Represent the Cube as a Group**
   - The Rubik's Cube can be modeled as a mathematical group (the Rubik's Cube Group), where:
     - Elements are cube states (permutations of stickers).  
     - The group operation is the composition of moves.  
   - Generators of the group are the six basic moves (e.g., U, D, F, B, L, R).  

*Mathematical Principle*: Group theory provides a framework for analyzing the cube's symmetries and move sequences.

#### **2. Use Solver Algorithms**
   - Employ **Thistlethwaite’s Algorithm** or **Kociemba's Algorithm**, which break the problem into subgoals by reducing the cube to subgroups.  
   - Example stages:
     1. Solve edges and corners independently.  
     2. Reduce the cube to a subgroup where only quarter turns are allowed.  
     3. Solve the cube completely.  

*Mathematical Principle*: Subgroup decomposition simplifies solving by restricting move possibilities.

#### **3. Optimality via Breadth-First Search (BFS)**
   - Use **BFS** to explore all possible move sequences, starting from the scrambled state, until the solved state is reached.  
   - This guarantees the minimal number of moves.  

*Mathematical Principle*: Graph theory ensures optimality by finding the shortest path in the state space.

#### **4. Output Moves with Explanations**
   - For each move, explain how it relates to group theory concepts (e.g., permutations, commutators).  
   - Example:  
     - **Move U**: A 90° clockwise turn of the upper face.  
     - **Math**: This is a generator of the Rubik's Cube Group, representing a specific permutation of stickers.  

---

### Complexity and Limitations
- **Complexity**: The Rubik's Cube Group has ≈4.3×10¹⁹ elements, making brute-force BFS infeasible.  
- **Optimization**: Use heuristic search (e.g., A*) or precomputed lookup tables (e.g., God's Number) to reduce computation.  

---

### Final Notes
This algorithm combines computational efficiency with group theory insights, ensuring both optimality and a deep understanding of the cube's mathematical structure.

In [25]:
# Updated with the latest Open Source model from OpenAI

groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "openai/gpt-oss-120b"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


Below is a **complete, high‑level design** of an algorithm that will (1) find a *minimum‑move* solution (the “God’s‑algorithm” solution) for any reachable Rubik’s‑Cube configuration, and (2) walk you through the **group‑theoretic ideas** that make the algorithm work.  
The description is deliberately language‑agnostic; you can implement it in C++, Python, Rust, etc., using the same mathematical building blocks.

---

## 1.  Why “minimum‑move” is hard – a quick group‑theory recap

| Concept | What it means for the cube | Why it matters for optimal solving |
|---------|---------------------------|-----------------------------------|
| **Rubik’s‑Cube group** 𝔾 | All reachable permutations of the 48 movable stickers. | |𝔾| = 43 523 616 218 400 ≈ 4.35 × 10¹⁹ – far too many to enumerate naïvely. |
| **Generators** | The six face‑quarter‑turn moves {U, U′, U2, R,…, B2}. | Any element (scramble) can be written as a word in these generators. |
| **Cosets & sub‑groups** | Typical two‑phase solvers work in a chain 𝔾 ⟶ H₁ ⟶ H₂ ⟶ … ⟶ {id}. | By forcing the search to stay inside a subgroup we dramatically shrink the branching factor. |
| **Parity, orientation & permutation constraints** | - Corner orientation sum = 0 (mod 3)  <br>- Edge orientation sum = 0 (mod 2)  <br>- Permutation parity of corners = parity of edges. | Guarantees that the set of legal states is a proper *subgroup* of the full permutation group; we never need to consider illegal configurations. |
| **Symmetry (cube automorphisms)** | 48 rotational symmetries + 24 mirror symmetries (if you allow them). | All positions that are equivalent under a symmetry have the same distance to the solved state – we can store one entry for many states. |
| **God’s Number = 20** | Proven (2010) by exhaustive computer search that every legal state can be solved in ≤ 20 quarter‑turns. | Any optimal algorithm must search at most depth‑20, but the branching factor (≈ 12) still yields ≈ 12²⁰ ≈ 10²¹ nodes – impossible without heavy pruning. |

The **core mathematical insight** is *coset decomposition*: we split 𝔾 into a handful of nested sub‑groups whose indices are modest (≈ 10⁴–10⁶). Inside each coset we can pre‑compute an **admissible heuristic** (a lower bound on distance) that is *perfect* for that sub‑problem. The heuristics are stored in **pattern‑databases** (also called *pruning tables*). When combined with an **IDA\*** (Iterative Deepening A*) search that respects symmetries, we can guarantee that the first solution found is optimal.

---

## 2.  High‑level algorithm outline

```
Algorithm OptimalRubikSolve(scramble):
    1. Encode the scramble as a CubeState object.
    2. Pre‑compute (once, offline) all pruning tables for the chosen subgroup chain.
    3. Run an IDA* search with depth limit L = 0,1,2,… until a solution is found:
          a) Use the pruning tables to compute a lower‑bound h(state).
          b) If current_depth + h(state) > L → prune this branch.
          c) Expand only moves that respect the subgroup‑phase constraints
             (e.g. Phase‑1 moves keep you inside H1, Phase‑2 moves keep you inside H2,…).
          d) Apply symmetry reduction: canonicalise the state under the cube’s
             rotation group and skip duplicate branches.
    4. Return the sequence of moves discovered when L reaches the true distance.
```

The algorithm is *exact* because:

* The pruning tables are **admissible** (never overestimate) and often **tight** (often equal to the true distance inside the coset).  
* IDA* explores states in order of increasing `depth + h`, so the first solution found at depth `L` is guaranteed to be the shortest possible.  
* The search depth never needs to exceed 20 (God’s Number).

---

## 3.  Detailed components

### 3.1.  Cube representation

| Piece type | Encoding | Size |
|------------|----------|------|
| Corners (8) | permutation index 0‑7 (3 bits each) + orientation 0‑2 (2 bits each) | 3 bits × 8 + 2 bits × 8 = 40 bits |
| Edges   (12) | permutation index 0‑11 (4 bits each) + orientation 0‑1 (1 bit each) | 4 bits × 12 + 1 bit × 12 = 60 bits |
| Total | 100 bits (fits in two 64‑bit words) | – |

A `CubeState` object stores these two 64‑bit words and provides:

* `apply(move)` – returns a new state after a quarter‑turn.  
* `canonical()` – returns the lexicographically smallest state among all 24 rotations (symmetry reduction).  
* `hash()` – for memoisation if needed.

### 3.2.  Sub‑group chain (the “two‑phase” skeleton)

The classic chain (used by Kociemba’s solver) is:

1. **𝔾** – whole group.  
2. **H₁** – all states with **edges oriented** and **corners oriented** (i.e. all orientations correct).  
   *Index* |𝔾 : H₁| ≈ 2 × 3⁷ × 2¹¹ ≈ 1.8 × 10⁸.  
3. **H₂** – additionally, **edges are in the correct slice** (M-slice edges are in M‑slice).  
   *Index* |H₁ : H₂| = 3⁴ = 81.  
4. **H₃** – finally, all pieces are permuted correctly (the solved state).  
   *Index* |H₂ : H₃| = 12! · 8! / (2 · 3) ≈ 5 × 10⁹ (but we never need a table for this last step; we finish with a simple BFS).

**Why this chain?**  
Each transition reduces the branching factor dramatically:

* In Phase 1 we may use **any** of the 18 face moves, but we restrict the *search* to keep the orientation constraints satisfied.  
* In Phase 2 we restrict to a **subset of moves** (U, U′, U2, D, D′, D2, R2, L2, F2, B2) that never disturb the already‑fixed slices.  
* In Phase 3 we finish with a simple “solve permutation” problem that can be tackled by a small lookup table (or a short BFS because the remaining depth ≤ 7).

### 3.3.  Pattern‑databases (pruning tables)

For each coset we store a *distance‑to‑goal* table that is **exact** for that sub‑problem. The most common tables are:

| Table | Encodes | Size (bytes) | How it is built |
|-------|---------|--------------|-----------------|
| **Corner‑orientation table** | 3⁷ = 2 187 possible orientations (the 8‑th is forced) | ≈ 2 KB | BFS from solved state, using only moves that preserve edge orientation. |
| **Edge‑orientation table** | 2¹¹ = 2 048 possible edge‑orientations | ≈ 2 KB | Same technique, moves preserve corner orientation. |
| **Slice‑permutation table** | 12 choose 4 = 495 possible placements of the 4 M‑slice edges | ≈ 500 KB (after symmetry compression) | BFS inside H₁, using only slice‑preserving moves (U/D turns). |
| **Corner‑permutation (reduced)** | 8! / 3 = 13 440 (quotient by 3‑cycle symmetry) | ≈ 14 KB | BFS inside H₂ (now all orientations fixed). |
| **Edge‑permutation (reduced)** | 12! / 2 = 239 500 800 (quotient by parity) – stored as a *combined* table with corners to keep size < 10 MiB. | ~ 10 MiB | BFS inside H₂, using only moves that keep slice correct. |

**Construction**  
All tables are generated **once** with a standard breadth‑first search from the solved state, *restricted to the allowed move set for that phase*. Because the state space of each table is at most a few million entries, the generation finishes on a modern PC in minutes.

**Admissibility proof**  
Each table stores the *exact* distance in its restricted group; therefore it is never larger than the distance in the full group, i.e. `h(state) ≤ true_distance(state)`. Adding the tables (taking the max) yields a heuristic that never overestimates the true distance.

### 3.4.  Symmetry reduction

The cube has 24 rotational symmetries (the orientation group of the cube). If we store the pruning tables **modulo symmetry**, the entry for a state and all its 24 rotated twins are identical. We therefore:

* **Canonicalise** every state before looking it up: apply the 24 rotations, pick the smallest binary encoding.  
* **Store** only one entry per orbit in the pruning tables (the tables are built that way from the start).  

Result: a typical pruning entry’s *effective* size shrinks by a factor ≈ 24, cutting both memory and search time.

### 3.5.  IDA* search with phase‑aware move generation

**Iterative Deepening A\*** works like:

```
for bound = heuristic(start) … 20:
    if dfs(start, 0, bound) succeeds → return solution
```

`dfs(state, g, bound)`:

```
h = heuristic(state)                 // max of all relevant pruning values
if g + h > bound: return false
if state == solved: return true
for each move m in AllowedMoves(state):
    next = state.apply(m)
    // optional: prune symmetric branches by forbidding the inverse move
    if dfs(next, g+1, bound):
        record m in solution stack
        return true
return false
```

**AllowedMoves** depends on the phase:

| Phase | Move set | Reason |
|-------|----------|--------|
| 1 | {U, U′, U2, R, R′, R2, F, F′, F2, L, L′, L2, B, B′, B2, D, D′, D2} (all 18) | Must keep orientation constraints satisfied; we *check* after each move that orientations are still “solved”. |
| 2 | {U, U′, U2, D, D′, D2, R2, L2, F2, B2} (10) | These moves never disturb the slice edges, so we stay inside H₂. |
| 3 | {U, U′, U2, D, D′, D2, R, R′, R2, L, L′, L2, F, F′, F2, B, B′, B2} (any) but depth is ≤ 7 | At this point only permutations remain; a tiny BFS or a pre‑computed “corner‑edge‑perm” table finishes it. |

The **inverse‑move pruning** (never immediately undo the previous turn) halves the branching factor.

### 3.6.  Guarantee of optimality

* **Bound** never exceeds 20 (God’s Number).  
* **Heuristic** is admissible → IDA* expands nodes in non‑decreasing order of `g + h`.  
* As soon as a solution is found at depth `L`, any other solution must have length ≥ `L` (otherwise it would have been discovered earlier).  

Thus the algorithm returns a *minimum‑move* (quarter‑turn) solution.

---

## 4.  Putting it all together – Pseudo‑code

```python
# --------------------------------------------------------------
# 1.  GLOBAL DATA (built once, then reused for every solve)
# --------------------------------------------------------------
prune_co =   load_table('corner_orientation.tbl')
prune_eo =   load_table('edge_orientation.tbl')
prune_slice =load_table('slice_permutation.tbl')
prune_cperm =load_table('corner_perm.tbl')
prune_eperm =load_table('edge_perm.tbl')
# all tables are stored in their canonical (symmetry‑reduced) form

# --------------------------------------------------------------
# 2.  Helper: lower bound heuristic for a given state
# --------------------------------------------------------------
def heuristic(state):
    # each table returns an integer distance in its sub‑group
    h1 = prune_co[state.corner_orientation_index()]
    h2 = prune_eo[state.edge_orientation_index()]
    h3 = prune_slice[state.slice_permutation_index()]
    # after phase‑1 we are guaranteed h1 == h2 == 0, but we keep them
    # for the early levels they give a tighter bound.
    # In phase‑2 we also use permutation tables:
    h4 = prune_cperm[state.corner_perm_index()]
    h5 = prune_eperm[state.edge_perm_index()]
    return max(h1, h2, h3, h4, h5)          # admissible (never overestimates)

# --------------------------------------------------------------
# 3.  IDA* core
# --------------------------------------------------------------
def ida_star(start):
    bound = heuristic(start)
    while bound <= 20:               # God's number
        path = []
        if dfs(start, 0, bound, path, None):
            return path               # sequence of moves (optimal)
        bound += 1
    raise RuntimeError("Should never happen – cube is unsolvable?")

def dfs(state, g, bound, path, last_move):
    h = heuristic(state)
    if g + h > bound:
        return False
    if state.is_solved():
        return True

    # Determine which phase we are in from the current depth g
    if g < PHASE1_LIMIT:            # e.g. 0..7 – we stay in H1
        moves = ALL_FACE_MOVES
    elif g < PHASE2_LIMIT:          # e.g. 8..14 – stay in H2
        moves = SLICE_PRESERVING_MOVES
    else:
        moves = ALL_FACE_MOVES       # final permutation phase

    for m in moves:
        if last_move and m.is_inverse_of(last_move):
            continue                # prune immediate back‑track
        nxt = state.apply(m)

        # Symmetry canonicalisation (optional, speeds up look‑ups)
        nxt_canon = nxt.canonical()

        # Early pruning using the admissible heuristic
        if g + 1 + heuristic(nxt_canon) > bound:
            continue

        path.append(m)
        if dfs(nxt_canon, g+1, bound, path, m):
            return True
        path.pop()
    return False
```

*`PHASE1_LIMIT`* and *`PHASE2_LIMIT`* are chosen empirically (most optimal solutions have ≤ 7 moves in Phase 1, ≤ 7 in Phase 2, ≤ 6 in Phase 3).  
If a particular scramble is known to require more moves in Phase 1, the algorithm automatically expands the bound and continues – it never misses a solution because the heuristic never over‑estimates.

---

## 5.  How the mathematics guarantees correctness

1. **Group structure** – The set of all reachable cube configurations forms a finite group 𝔾 generated by the six face turns. The subgroup chain  
   `𝔾 ⊃ H₁ ⊃ H₂ ⊃ {id}` is a *normal series* (each subgroup is normal in the previous one).  
   By the **Fundamental Theorem of Homomorphisms**, the coset spaces 𝔾/H₁, H₁/H₂, H₂/{id} are themselves groups whose orders are the indices given above. Working inside a coset means we only have to consider the *quotient* group’s generators, dramatically reducing the branching factor.

2. **Coset pruning tables** – For a subgroup K, a BFS from the identity explores exactly the **Cayley graph** of K with respect to the allowed generators. The distance recorded in the table is the **word metric** (minimum number of generators) in that graph. Since K ≤ 𝔾, any word in K is also a word in 𝔾, so the stored distance is a **lower bound** for the true distance in 𝔾.

3. **Admissible heuristic** – Taking the maximum of several lower bounds yields another lower bound (max preserves ≤). Thus `h(state) ≤ d_𝔾(state, solved)` for every state.

4. **IDA\* optimality** – IDA* expands nodes in order of non‑decreasing `f = g + h`. Because `h` never overestimates, the first time the search reaches the solved state we have `g = d_𝔾(start, solved)`. This is a standard theorem from heuristic search.

5. **Symmetry reduction** – The cube’s rotation group R (|R| = 24) acts on 𝔾 by automorphisms. The distance function is invariant under R (rotating a scramble does not change its distance to the solved state). Therefore we may replace each state by the *orbit representative* (canonical form) without affecting the optimal distance. This reduces the effective state space by a factor of up to 24, which is why the tables fit in a few megabytes.

6. **God’s Number** – The exhaustive computer proof (2010) showed that the *diameter* of the Cayley graph of 𝔾 under the quarter‑turn metric is 20. Consequently, the outer loop of IDA* never needs to increase `bound` beyond 20, guaranteeing termination in a finite (and modest) number of iterations.

---

## 6.  Practical notes & extensions

| Issue | Practical tip |
|-------|----------------|
| **Memory** | Even with symmetry compression the biggest pruning table (combined edge‑/corner‑perm) is ≈ 10 MiB. This easily fits in RAM on any modern machine. |
| **Speed** | A well‑optimised C++ implementation solves a random scramble in < 0.2 s on a 3 GHz CPU. Python with `numpy`/`numba` can still finish under 2 s. |
| **Move metric** | The design above uses the **quarter‑turn metric** (each 90° turn = 1). For the **face‑turn metric** (180° counts as 1) simply replace the move set and rebuild the tables accordingly. |
| **Parallelisation** | The BFS that builds the tables can be split across cores; the final IDA* search is inherently sequential but the outer depth‑bound loop can be parallelised by launching separate threads with different bounds. |
| **Learning‑based heuristics** | Recent research (e.g., DeepCubeA) trains neural networks to predict `h`. Those heuristics are *not* guaranteed admissible, so they give fast *near‑optimal* solutions but lose the optimality guarantee. Our design deliberately stays within pure group‑theoretic, provably‑optimal territory. |
| **Extension to other puzzles** | The same coset‑decomposition + pattern‑database + IDA* framework works for any puzzle whose state space is a finite group with a known generator set (e.g., 4×4×4 cube, Megaminx). |

---

## 7.  Summary

1. **Mathematics** – By viewing the Rubik’s Cube as a finite permutation group, we break the search into a chain of sub‑groups, each with a tractable quotient. Symmetry and parity constraints shrink the effective state space dramatically.

2. **Data structures** – Exact *pruning tables* (pattern databases) give a perfect lower bound for each sub‑problem. They are built once offline using BFS in the restricted Cayley graphs.

3. **Search algorithm** – An **IDA\*** driver, guided by the max of the pruning values, explores the state space in increasing `g+h`. Because the heuristic is admissible and the outer bound never exceeds **20**, the first solution found is provably optimal.

4. **Result** – The algorithm returns a **minimum‑move (≤ 20)** solution for any legal scramble, and the whole process is fully explained by concrete group‑theoretic concepts (generators, cosets, normal series, automorphisms) and classic AI search theory (admissible heuristics, IDA*).

You now have everything needed to:

* **Implement** a fast, optimal Rubik’s‑Cube solver.  
* **Teach** the underlying algebra: how sub‑group chains and symmetry turn a 10¹⁹‑size problem into a handful of megabyte tables and a depth‑20 search.  

Happy coding, and enjoy watching the beautiful interplay of abstract algebra and algorithmic search turn a scrambled cube into a solved one in the fewest possible twists!

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [26]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing an efficient algorithm to solve a Rubik's Cube involves understanding group theory, particularly the symmetries of the cube and the group operations that can be applied.

1.  Cubic representation:
    The Rubik's Cube is often represented with its 43 faces arranged by a matrix with six layers.
2.  Symmetry Group:
    Each of the 6 faces has two symmetric points on both sides. These symmetries produce an ordered group \((S_3)\) containing 12 distinct permutations. This symmetry is essential in forming the move, which helps determine whether it's one step back from the start or not.
3.  Number of Moves: The minimum number of moves required to solve a Rubik's Cube varies depending on the starting configuration.

Given that every possible configuration (i.e., cube permutation) can be reached by any starting arrangement through at most \(4, 7\) and some configurations as low as five algorithms, we will create an efficient algorithm.

**Algorithm**

1.  **Find the center pieces:** 
    \*   The middle face piece
    *   Top middle layer corner piece
2.  **First two corners piece:**
3.  **Three corner edge piece and top middle layer edge piece**
4.  **All edges are fixed and only top middle layer edge pieces move.**

Here's the modified algorithm based on this strategy:

1.  If center piece is not white:
    -   (R, U2, R', U')
              The "U' L2 U" sequence will return first corner into its correct position.
              Step number is 3+2: U1(L)=Y
2.  Two corners are the same color. 
     *   If top middle layer corner piece is yellow:
    -   (R, U)
     Step Number: 6. R'U2U
    *   First two corner pieces can be swapped in the next step.
3.  There is one up-middle edge piece and a lower down-edge.
     *   (R) L=U(R)=(U'(S))L'
       \*   The center layer edge moves to top middle layer edge.

If it takes more than 20 steps, swap the first two corner pieces in each sequence for the minimum number of moves required.

4.  **The rest are solved using the "First Two Corners" algorithm:**
     *   Step Number :3
    -   R, U2 (first down edge), R’
    -   The center layer edge can be moved to top middle layer edge with U(R) L.
5.  Use two sequences of three piece moves.

6.  Do **Two Color Corners:**
     *   If the color first corner contains yellow
        -   Move corner to top edge through the white up corner
    *   If down-color is yellow - move back through up in same step

7.  To change middle and right layers use "3 piece moves"

8.  Now we're left with no two color corners.

9.  Solve the rest in the next two moves

In [27]:
# So where are we?

print(competitors)
print(answers)


['gpt-5-nano', 'FreeLLM API (apifreellm.com)', 'gemini-2.5-flash', 'FreeLLM API (apifreellm.com)', 'openai/gpt-oss-120b', 'llama3.2']
['Short answer: Yes. You can build an optimal Rubik’s Cube solver that guarantees a minimum-move solution for any scramble, and you can accompany it with a solid explanation of the group-theory ideas that make optimality possible. In practice this is done with a combination of pattern databases (PDBs) and a search procedure that is guaranteed not to overestimate the remaining distance (e.g., IDA* with an admissible heuristic). Below is a concrete high-level design you can implement or study, plus the mathematical background that makes it work.\n\n1) What “minimum-move solver” means in Rubik’s Cube terms\n- The cube group G is generated by the six face turns (U, D, L, R, F, B). A solution is a shortest sequence of these moves that transforms a given scramble to the solved state.\n- The “minimum number of moves” depends on the move metric you use. The comm

In [28]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


Competitor: gpt-5-nano

Short answer: Yes. You can build an optimal Rubik’s Cube solver that guarantees a minimum-move solution for any scramble, and you can accompany it with a solid explanation of the group-theory ideas that make optimality possible. In practice this is done with a combination of pattern databases (PDBs) and a search procedure that is guaranteed not to overestimate the remaining distance (e.g., IDA* with an admissible heuristic). Below is a concrete high-level design you can implement or study, plus the mathematical background that makes it work.

1) What “minimum-move solver” means in Rubik’s Cube terms
- The cube group G is generated by the six face turns (U, D, L, R, F, B). A solution is a shortest sequence of these moves that transforms a given scramble to the solved state.
- The “minimum number of moves” depends on the move metric you use. The common one is the quarter-turn metric (QTM): a 90° turn counts as 1, a 180° turn counts as 2. In that metric, the maxima

In [29]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [30]:
print(together)

# Response from competitor 1

Short answer: Yes. You can build an optimal Rubik’s Cube solver that guarantees a minimum-move solution for any scramble, and you can accompany it with a solid explanation of the group-theory ideas that make optimality possible. In practice this is done with a combination of pattern databases (PDBs) and a search procedure that is guaranteed not to overestimate the remaining distance (e.g., IDA* with an admissible heuristic). Below is a concrete high-level design you can implement or study, plus the mathematical background that makes it work.

1) What “minimum-move solver” means in Rubik’s Cube terms
- The cube group G is generated by the six face turns (U, D, L, R, F, B). A solution is a shortest sequence of these moves that transforms a given scramble to the solved state.
- The “minimum number of moves” depends on the move metric you use. The common one is the quarter-turn metric (QTM): a 90° turn counts as 1, a 180° turn counts as 2. In that metric, the 

In [31]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [32]:
print(judge)

You are judging a competition between 6 competitors.
Each model has been given this question:

Can you design an algorithm that not only solves any given Rubik's Cube in the minimum number of moves but also explains the underlying mathematical principles and group theory concepts that make the solution optimal?

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}

Here are the responses from each competitor:

# Response from competitor 1

Short answer: Yes. You can build an optimal Rubik’s Cube solver that guarantees a minimum-move solution for any scramble, and you can accompany it with a solid explanation of the group-theory ideas that make optimality possible. In practice this is done with a combination of pattern databases (PDBs) and a search procedu

In [33]:
judge_messages = [{"role": "user", "content": judge}]

In [49]:
# Judgement time!

openai = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
# model_name = "llama3.2"
# model_name = "google/gemini-2.5-flash-lite"

response = openai.chat.completions.create(model=model_name,
                                          messages=judge_messages)
results = response.choices[0].message.content
print(results)

{"results": ["5", "3", "1", "2", "4", "6"]}


In [50]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: openai/gpt-oss-120b
Rank 2: gemini-2.5-flash
Rank 3: gpt-5-nano
Rank 4: FreeLLM API (apifreellm.com)
Rank 5: FreeLLM API (apifreellm.com)
Rank 6: llama3.2


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>